In [ ]:
import numpy as np
from dadk.QUBOSolverCPU import *
from dadk.BinPol import *
from datetime import datetime
import pandas as pd

# Input Data

In [ ]:
# Input data 

# Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].values.tolist()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].values.tolist()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].values.tolist()

n_assets = len(asset_quantity)

# Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].values.tolist()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].values.tolist()

n_accounts = len(account_exposure)

# Haircuts
haircuts = (pd.read_csv("data/sample_haircuts.csv", header=None).values.tolist())

#Single Limits
single_limits = (pd.read_csv("data/sample_single_limits.csv", header=None).values.tolist())

# Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

 ## Penalty Terms


In [ ]:
# Penalty terms
lambda_cost_fn = 1e5
lambda_consistency1, lambda_consistency2 = 1, 1
lambda_exposure1, lambda_exposure2 = 1, 5e2

#lambda_single_lim1, lambda_single_lim2 = 1,1

# Constructing the QUBO

In [ ]:
# n-bit binary variables
n_binary = 7
binary_coeff = []
max_binary_value = (2**n_binary) - 1

for b in range(1,n_binary+1):
    aux_term = (2**(b-1) * 1) / max_binary_value
    binary_coeff.append(aux_term)

single_limits_fractional = np.zeros_like(single_limits) 
single_limits_num_bits = np.zeros((n_assets,n_accounts), dtype=int) 


for i in range(n_assets):
    for j in range(n_accounts):
        single_limits_fractional[i][j] = single_limits[i][j]/asset_quantity[i]
        aux_variable = int(np.floor(np.log2((single_limits_fractional[i][j])*max_binary_value)))
        single_limits_num_bits[i][j] = n_binary if aux_variable > n_binary else aux_variable

binary_variables = []

for i in range(n_assets):
    for j in range(n_accounts):
        binary_variables.append(BitArrayShape(name=f'bin_vars{i},{j}', shape=(1, 1, single_limits_num_bits[i][j]), axis_names=['Asset', 'Account', 'Binary']))


var_shape_set = VarShapeSet(*binary_variables)


In [ ]:
# Constructing QUBO equation

# Objective function
scale_cost_fn = 1000

# Objective function
cost_function = BinPol(var_shape_set)
for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(single_limits_num_bits[i][j]):
            cost_function.add_term(cost_factor_matrix[i][j]*binary_coeff[k],((f'bin_vars{i},{j}', 0, 0, k),))
cost_function.multiply_scalar(scale_cost_fn)
print(cost_function)

In [ ]:
#consistency requirement
con1 = BinPol(var_shape_set)
con2 = BinPol(var_shape_set)
scale_con1 = 1000
scale_con2 = 100000

con_constraint = BinPol(var_shape_set)
for i in range(n_assets):
    aux_term = BinPol(var_shape_set)
    for j in range(n_accounts):
        for k in range(single_limits_num_bits[i][j]):
            aux_term.add_term(binary_coeff[k],((f'bin_vars{i},{j}', 0, 0, k),))
        aux_term.add_term(-1,())
    con1.add(aux_term)
    aux_term.power(2)
    con2.add(aux_term)
verify_con = con1 + con2


con1.multiply_scalar(lambda_consistency1)
con2.multiply_scalar(lambda_consistency2)



con_constraint = con1*scale_con1 + con2*scale_con2
print(con_constraint)


In [ ]:
# Exposure requirements
exposure1  = BinPol(var_shape_set)
exposure2 = BinPol(var_shape_set)
scale_exposure1 = 1e4
scale_exposure2 = 1e7
norm_exposure = max(np.array(asset_quantity)*np.array(asset_value))*64/max_binary_value

for j in range(n_accounts):
    aux_term = BinPol(var_shape_set)
    for i in range(n_assets):
        for k in range(single_limits_num_bits[i][j]):
            aux_term.add_term(binary_coeff[k]*asset_quantity[i]*asset_value[i]*haircuts[i][j]/100,((f'bin_vars{i},{j}', 0, 0, k),))
    aux_term.add_term(-account_exposure[j],())
    exposure1.add(aux_term)
    aux_term.power(2)
    exposure2.add(aux_term)

verify_exposure = -exposure1 + exposure2

exposure1.multiply_scalar(lambda_exposure1)
exposure2.multiply_scalar(lambda_exposure2)



exposure_requirements = -exposure1*scale_exposure1/norm_exposure + exposure2*scale_exposure2/(norm_exposure**2)



In [ ]:
# Final QUBO
final_equation = (cost_function*lambda_cost_fn + con_constraint + exposure_requirements)


# Run Solver

In [ ]:
solver = QUBOSolverCPU(
   
    number_iterations    = 50000,               # total number of itrations per run
    number_runs          = 4,                 # number of stochastically independant runs
    temperature_start    = 5000,               # start temperature for annealing as float value
    temperature_end      = 10,                 # end temperature for annealing as float value 
    temperature_mode     = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_decay    = 0.0095,             # see temperature_mode 0
    temperature_interval = 1,                  # see temperature_mode 0
    offset_increase_rate = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    graphics             = True                # create data for graphics output
)


In [ ]:
# Run solver
start=datetime.now()
solution_list = solver.minimize(final_equation)

# Get results
solution = solution_list.get_minimum_energy_solution()
configuration = solution.configuration
print("Cost function:", cost_function.compute(configuration)/1000)
print("Consistency constraint:",verify_con.compute(configuration))
print("Exposure:", verify_exposure.compute(configuration))

# Verify Results

In [ ]:
my_bit_array = []

for i in range(n_assets):
    for j in range(n_accounts):
        my_bit_array.append(solution.extract_bit_array(f"bin_vars{i},{j}"))


exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage and Verification of Single Limits=====")
counter = 0
for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            bit += str(my_bit_array[counter][0][0][k])
            decimal +=(2**k)*my_bit_array[counter][0][0][k]
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        if (decimal/max_binary_value)*asset_quantity[i] > single_limits[i][j]: 
            print(f"Single limit constraint satisfied: {((decimal/max_binary_value)*asset_quantity[i]) <= single_limits[i][j]}")
            print(f"Quantity assigned = {(decimal/max_binary_value)*asset_quantity[i]:0.2f}, Single limit is {single_limits[i][j]:0.2f}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]/100
        counter += 1
    allocation_solution[i] = asset
    


In [ ]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

In [ ]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i],:} required")


In [ ]:
print(datetime.now()-start)

# Export to .csv

In [ ]:
# Export to .csv

Q_value_unbalanced_fujitsu = np.zeros(shape=(n_assets,n_accounts))

counter = 0
for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            bit += str(my_bit_array[counter][0][0][k])
            decimal +=binary_coeff[k]*my_bit_array[counter][0][0][k]*max_binary_value
        asset += decimal*(100/max_binary_value)
        Q_value_unbalanced_fujitsu[i][j] = decimal/max_binary_value
        counter += 1


#print(Q_value_unbalanced_fujitsu)
print(f"Objective value: {sum(sum(Q_value_unbalanced_fujitsu*cost_factor_matrix))}")
#Uncomment the line below to save the Data to csv

# pd.DataFrame(Q_value_unbalanced_py_qubo).to_csv("Q_value_unbalanced_fujitsu_no_single_lim3.csv", header=None, index=None)